# Script de python sencillo para descargar datos de EAGLE 

#### Antes de empezar: una herramienta útil para descargar datos de EAGLE desde Python, es el paquete eagleSQLtools.

#### Repositorio del paquete: https://github.com/kyleaoman/eagleSqlTools

#### Para instalarlo, en una terminal correr el siguiente comando:

#### pip install eaglesqltools

#### Importar paquetes necesarios. Agregar paquete si hace falta

In [1]:
import matplotlib.pyplot as plt
import eagleSqlTools as sql
from astropy.io import ascii
from astropy.table import Table
import numpy as np

#### Datos para la descarga (usuario, contraseña, simulación, snapnum, fragmentos de la query)

In [23]:
# Usuario y contraseña
usr='cht015'
pwd='BH457tfj'

# =========================================================================================

# Simulación y snapnum deseado
simu='NoAGNL0050N0752'
snap = 28
# aperture = 30

# =========================================================================================

# Carpeta donde guardar archivo. Dejar un caracter 'vacío' para descargar los datos
# en la misma carpeta desde donde se ejecuta la notebook. Datos guardados/Facultad/Tesis/Datos/Proyecto Altas Masas
download_folder='/home/ramiro/Datos guardados/Facultad/Tesis/Datos/Proyecto Altas Masas/'

# Nombre del archivo que guardará los datos descargados
filename='catalogo_proyecto_altas_masas_'+simu+'_snap_'+str(snap)+'.dat'

# =========================================================================================
# Cosas para armar la query

# Tablas y Alias de las tablas desde donde quiero descargar datos
# Agregar/quitar según sea necesario
tables=[
        'Subhalo',
        'MorphoKinem',
#         ,'FOF'
       ]

aliases=[
         'sub'
         ,'mor'
#          ,'fof' 
        ]

# --------------------------------------------------------------------------------

# Columnas a seleccionar de cada tabla (agregar/quitar según sea necesario)
columns=[
          
          [
          'GalaxyID','SnapNum','GroupID','SubGroupNumber','Stars_Mass','SF_Hydrogen','SF_Oxygen','SF_Mass','NSF_Mass','BlackHoleMass','StarFormationRate'
          ]
          ,[
          'KappaCoRot '
          ]
#           ,[
#           'Group_M_Crit200'
#           ]
         ]

# --------------------------------------------------------------------------------

# Condiciones sobre las columnas de las tablas a seleccionar
# NO AGREGAR ACÁ EL ALIAS DE LA TABLA! Se agregará después
# Agregar/quitar según sea necesario
# Cada lista de la siguiente lista establece condiciones para la tabla
# a seleccionar (respetando el orden en que se definieron las tablas).
# Si no se quieren condiciones sobre alguna tabla, dejar la correspondiente
# lista como vacía.

conditions=[
            ['SnapNum='+str(snap),'SF_Oxygen>0','SF_Hydrogen>0','SubGroupNumber = 0']
           ]

# --------------------------------------------------------------------------------


#### Construcción de las distintas partes de la query

In [24]:
# Armo la sentencia SELECT 
select=''
for alias,col in zip(aliases,columns):
    select=select+(','.join([alias+'.'+name for name in col]))+','
select=select[:-1]   # Esto es para borrar una última coma 'molesta'

# --------------------------------------------------------------------------------

# Armo la sentencia FROM
from_table=''
for tab,alias in zip(tables,aliases):
    from_table=from_table+simu+'_'+tab+' as '+alias+','
from_table=from_table[:-1]   # Esto es para borrar una última coma 'molesta'

# --------------------------------------------------------------------------------

# Armo la sentencia WHERE
# Condiciones para unir tablas (igualdad de GalaxyIDs)
join_conditions=''
# Lo sigueinte solo se ejecutará si hay más de una tabla
if len(tables)>1:
    for k in range(len(aliases)-1):
        join_conditions=(join_conditions+
                         aliases[k]+'.GalaxyID='+aliases[k+1]+'.GalaxyID'+' and ')
    join_conditions=join_conditions[:-5]

where=''    
for alias,condit in zip(aliases,conditions):
    if len(condit)>0:
        where=where+(' and '.join(alias+'.'+cond for cond in condit))+ ' and '
        if len(aliases)==1:
            where=where[:-4] # Esto es para borrar un 'and' molesto
where=where+join_conditions

In [25]:
# Testeo de la query
print('SELECT')
print(select)
print()
print('FROM')
print(from_table)
print()
print('WHERE')
print(where)

SELECT
sub.GalaxyID,sub.SnapNum,sub.GroupID,sub.SubGroupNumber,sub.Stars_Mass,sub.SF_Hydrogen,sub.SF_Oxygen,sub.SF_Mass,sub.NSF_Mass,sub.BlackHoleMass,sub.StarFormationRate,mor.KappaCoRot 

FROM
NoAGNL0050N0752_Subhalo as sub,NoAGNL0050N0752_MorphoKinem as mor

WHERE
sub.SnapNum=28 and sub.SF_Oxygen>0 and sub.SF_Hydrogen>0 and sub.SubGroupNumber = 0 and sub.GalaxyID=mor.GalaxyID


In [26]:
# where = 'sub.SnapNum='+str(snap)+' and sub.Stars_Mass>=1e10 and sub.SF_Oxygen>0 and sub.SF_Hydrogen>0 and sub.GalaxyID=mor.GalaxyID and sub.GroupID=fof.GroupID'

In [27]:
# print(where)

#### Conexión a la base de datos y descarga

In [28]:
# Conectarse a la base de datos
con = sql.connect(usr,password=pwd)

# Query en SQL
query = 'SELECT '+select+' FROM '+from_table+' WHERE '+where

# Execute query 
exquery = sql.execute_query(con, query)

# List of column names of downloaded data
colnames=(exquery.view(np.recarray).dtype.names)

# Dictionary of data
mytable={}
for name in colnames:
    mytable[name]=exquery[name]

# dictionary={key1:value,key2:value,....}
    

Exception: [b'#SQL= SELECT sub.GalaxyID,sub.SnapNum,sub.GroupID,sub.SubGroupNumber,sub.Stars_Mass,sub.SF_Hydrogen,sub.SF_Oxygen,sub.SF_Mass,sub.NSF_Mass,sub.BlackHoleMass,sub.StarFormationRate,mor.KappaCoRot  FROM NoAGNL0050N0752_Subhalo as sub,NoAGNL0050N0752_MorphoKinem as mor WHERE sub.SnapNum=28 and sub.SF_Oxygen>0 and sub.SF_Hydrogen>0 and sub.SubGroupNumber = 0 and sub.GalaxyID=mor.GalaxyID\n', b'#MAXROWS UNLIMITED\n', b'#QUERYTIMEOUT 1800 sec\n', b'#SQLSTATE S0002\n', b'#SQLERRORCODE 208\n', b"#SQLEXCEPTION =Invalid object name 'NoAGNL0050N0752_MorphoKinem'."]

#### Guardar los datos en un archivo ascii

In [ ]:
# Abrir el archivo donde se guardarán los datos
outf=open(download_folder+filename,'w')

# Transformar el diccionario a una tabla ascii
data_ascii=Table(mytable)

# Escribir los datos al archivo
data_ascii.write(outf,format='csv')

# Cerrar el archivo
outf.close()

#### EoP